<a href="https://colab.research.google.com/github/MubashirSaeed942/function_calling_LAngchain/blob/main/functionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [3]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY_1')

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-88acf91d-bcf2-4334-822d-b8d9b8804be5-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [13]:
from langchain_core.tools import tool
@tool
def multiply(a:int, b:int)-> int:
  """multiply both numbers.
    Args:
     a integer as first number
     b integer as second number    """
  return a*b

tools_list = [multiply]
model_with_tools = llm.bind_tools(tools_list)

result = model_with_tools.invoke('what is 2 multiply by 10')
result


AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 10.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-fea91bed-1552-447b-980a-c8a5364fb32f-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 10.0}, 'id': 'c996d3f6-f40c-4e6f-82ef-42484c2681aa', 'type': 'tool_call'}], usage_metadata={'input_tokens': 73, 'output_tokens': 3, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}})

In [14]:
from langchain.tools import Tool
def add (a:int, b: int) -> int:
  return a + b

math_tools = Tool(
    name = 'AdditionTool',
    func = lambda x: f"The result is : {add(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."

)



In [18]:
from langchain.agents import initialize_agent ,AgentType
tools = [math_tools]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [17]:
my_response = agent.run('what is 30 + 99')
print(my_response)




> Entering new AgentExecutor chain...
Thought: I need to add 30 and 99.  I can use the AdditionTool for this.
Action: AdditionTool
Action Input: 30 99
Observation: The result is : 129
Thought:I now know the final answer
Final Answer: 129


> Finished chain.
129


**ASSIGNMENT**

In [48]:
import requests

weather_api = '80d45b3bfdaa2b37dbdd282a17f3cefe'
base_url = 'https://api.openweathermap.org/data/2.5/weather'

def weather_check (city:str)->dict :
   """
    Function to fetch the weather for a given city using OpenWeather API.

    Args:
    - city (str): The name of the city to get the weather for.

    Returns:
    - dict: The weather data for the city.
    """

   complete_url = f"{base_url}?q={city}&appid={weather_api}"
   response = requests.get(complete_url)
   data = response.json()
   return data

# data1 = weather_check("Islamabad")
# print("Weather Data:", data1)
# if data1:
#     print("Valid Data:", data1.get('main', {}).get('temp', 'Not Found'))
# else:
#     print("No Data Available")






In [43]:
from langchain.tools import Tool
weather_tool = Tool(
    name='Weatherchecker',
    func= weather_check,
    description="Fetches weather information for a given city."
)


In [47]:
from langchain.agents import initialize_agent ,AgentType
tools = [weather_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [52]:
while True:
  city = input('Enter your City')
  if city.lower() == 'exit':
    break
  response = agent.run(f'what did you tell me about weather of {city}')
  response

Enter your CityExit
